In [107]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import campaign_monitor_modul as cm

import gspread
from fractions import Fraction

%load_ext autoreload
%autoreload 2

# Replace with your actual API key and client ID
api_key = 'W7KbXzk9fFsN+G3XrIemdIu156KSPq3Qh60OQFQw9oP0UPnDvfq8w2SH5plgeZnpzyA4Bc2QCKVpK9SB42n8ZO3Z4Cpefu2/HGULJneNoJZ2zza129wfcsOL35Fpoweq3ZcidTNu110I/HCNguRGUw=='
client_id = '1545f43bbfe89f34ea7cf143c36c5e9f'
campaign_id = '5489242c196e25e8772adf912ca66443'

# API endpoint
url = f'https://api.createsend.com/api/v3.2/clients/{client_id}/campaigns.json'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
# list_campaign = ['Vol_244',
#              'Vol_245']
# search_value = '|'.join(list_campaign)
# print(search_value)

Vol_244|Vol_245


In [108]:
search_value = 'Villa_Owners_W4Sep24_Karma_Community_Vol_244'
df_campaign = cm.get_campaign_list(api_key, client_id, search_value)
df_campaign

Campaigns List:


,Name,SentDate,TotalRecipients,CampaignID,Subject
0,Villa_Owners_W4Sep24_Karma_Community_Vol_244,2024-09-23 19:45:00,196,02c52eb08befd672824c436f67425270,Rugby Stars Battle Down Under


In [109]:
count = 0
frames = []

# mengeluarkan isi dataframe menggunakan perulangan
for index, row in df_campaign.iterrows():
        print(f"Retrieveing openers data {row['Name']}")

        # Mencari jumlah page openers dari campaign name
        globals()['get_openers_list%s' % count] = cm.get_data_list(api_key, row['CampaignID'], 'opens')

        # Mengambil semua data openers dari campaign name
        globals()['page_total%s' % count] = globals()['get_openers_list%s' % count]['page_total']

        # Jika page total lebih dari 0, langsung loop data
        if  globals()['page_total%s' % count] > 0:
                globals()['df_campaign_openers%s' % count] = cm.loop_data_list_page(api_key, row['CampaignID'], 'opens', globals()['page_total%s' % count])
                globals()['df_campaign_openers%s' % count]['Campaign Name'] = row['Name']
                frames.append(globals()['df_campaign_openers%s' % count])
        count += 1
if len(frames) > 0:
        df_campaign_openers = pd.concat(frames, ignore_index=True)
else:
        df_campaign_openers = pd.DataFrame({'Campaign Name': [''], 'Openers': ['']})   

Retrieveing openers data Villa_Owners_W4Sep24_Karma_Community_Vol_244


In [110]:
df_campaign_openers.to_excel(r'C:\Users\fajar\Documents\Python\Data\openers_all.xlsx', index=False)

In [111]:
df_campaign_openers.drop_duplicates(subset=['EmailAddress'], inplace=True)
# df_campaign_openers.to_excel(r'C:\Users\fajar\Documents\Python\Data\openers_unique.xlsx', index=False)

In [112]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1_O4_ZmZJe86gS_qiz7W0PZ2-USiVIFbVu6U7XWXCBUk'
SHEET_NAME = 'Test'
# SHEET_NAME = 'Sheet1'

In [113]:
def next_available_row(worksheet):
    str_list = worksheet.col_values(1)
    return str(len(str_list)+1)

In [114]:
spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

df_members = pd.DataFrame(rows)
df_members

,Email,Name
0,KarmaIndia.salesmanagers@karmagroup.com,Karma India. Sales Managers
1,johnandmel1@btinternet.com,John Windley
2,Anne.ginzel@gmail.com,Charlie Dove
3,Kevin.Matthews@afgonline.com.au,
4,Max.Goebl@gmx.de,
...,...,...
98,wbudiman@gmail.com,
99,wen.lu688@googlemail.com,
100,wkurniawan22@yahoo.com,
101,yuli.vkool@gmail.com,


In [115]:
next_row = int(df_members.shape[0])+1
next_row

104

In [116]:
df_update = df_campaign_openers['EmailAddress'].loc[~df_campaign_openers['EmailAddress'].isin(df_members['Email'])].copy()
df_update.reset_index(drop=True, inplace=True)

In [117]:
total_data = int(df_update.shape[0])
last_update_range = next_row+total_data
last_update_range

104

In [118]:
first_column = 'A'
last_column = 'A'

update_range = first_column+str(next_row)+':'+last_column+str(last_update_range)
update_range

'A104:A104'

In [119]:
# test_list = ['Anne.ginzel@gmail.com',
#  'Kevin.Matthews@afgonline.com.au']

In [120]:
# Jika ada data yang harus di update
if int(df_update.shape[0]) > 0:
    try:
        worksheet.update(update_range, [[value] for value in df_update.values.tolist()])
    except gspread.exceptions.APIError as e:
        print(e)
else:
    print("Tidak ada data yang harus diupdate")


Tidak ada data yang harus diupdate
